In [1]:
#%cd .. 
#!rm -rf NAS_LLM
!git clone -b features https://github.com/WpythonW/NAS_LLM.git
%cd NAS_LLM/
!uv pip install --system -q -r pyproject.toml

Cloning into 'NAS_LLM'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 221 (delta 16), reused 30 (delta 9), pack-reused 175 (from 1)
Receiving objects: 100% (221/221), 2.32 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (100/100), done.
/kaggle/working/NAS_LLM


In [1]:
%cd NAS_LLM/
!cat llm_opt_toolkit/llm_requester.py

/kaggle/working/NAS_LLM
from google import genai
from google.genai import types
from config import GEMINI_API_KEY
from pydantic import BaseModel

from google import genai
from google.genai import types
from google.genai.errors import ServerError
from pydantic import BaseModel
import time

def call_llm(prompt: str, ListConfigs: BaseModel, temperature: float = 1, thinking_budget: int = 2048, max_retries: int = 100):
    client = genai.Client(api_key=GEMINI_API_KEY)
    
    config = types.GenerateContentConfig(
        system_instruction="You are hyperparameter optimizer. Strictly follow rules in prompt.",
        response_mime_type="application/json",
        response_schema=ListConfigs,
        temperature=temperature,
        thinking_config=types.ThinkingConfig(thinking_budget=thinking_budget)
    )

    for attempt in range(max_retries):
        try:
            print('Attempting to call LLM, try', attempt + 1)
            response = client.models.generate_content(
                m

In [27]:
!pwd

/kaggle/working/NAS_LLM


In [28]:
%%writefile llm_opt_toolkit/optimization_journal.py

import os
import json
import pandas as pd
from datetime import datetime
import numpy as np
from config import JOURNAL_DIR

class Journal:
    def __init__(self, filename=None):
        os.makedirs(JOURNAL_DIR, exist_ok=True)
        
        if filename is None:
            filename = f'experiments_pred_{datetime.datetime.now()}.json'
        
        self.filepath = os.path.join(JOURNAL_DIR, filename)
        self.entries = self._load()
    
    def _load(self):
        if not os.path.exists(self.filepath):
            return []
        
        with open(self.filepath, 'r') as f:
            content = f.read().strip()
            if not content:
                return []
            return json.loads(content)
    
    def _save(self):
        with open(self.filepath, 'w') as f:
            json.dump(self.entries, f, indent=2, ensure_ascii=False)
    
    def add(self, config, measures, trial: int):
        if config.label_len >= config.seq_len:
            raise ValueError(f"label_len >= seq_len: {config.label_len} >= {config.seq_len}")
        
        entry = {
            'trial': int(trial),
            'timestamp': datetime.now().isoformat(),
            **{k: float(v) for k, v in measures.items()},
            **{k: int(v) if isinstance(v, (np.integer, np.int64)) else v 
               for k, v in config.__dict__.items()}
        }
        self.entries.append(entry)
        self._save()
    
    def get_history_table(self, last_n=1500) -> str:
        if not self.entries:
            return ""
        
        df = pd.DataFrame(self.entries[-last_n:]).drop(columns=['trial', 'timestamp'])
        
        top3_mse = df.nsmallest(4, 'mse_val')
        #top3_mae = df.nsmallest(3, 'mae')
        worst3_mse = df.nlargest(4, 'mse_val')
        #worst3_mae = df.nlargest(3, 'mae')
        
        tables = [
            "\n### История последних экспериментов",
            df.to_markdown(index=False, floatfmt=".4f"),
            "\n### Топ 3 по MSE",
            top3_mse.to_markdown(index=False, floatfmt=".4f"),
            #"\n### Топ 3 по MAE",
            #top3_mae.to_markdown(index=False, floatfmt=".4f"),
            "\n### Худшие 3 по MSE",
            worst3_mse.to_markdown(index=False, floatfmt=".4f"),
            #"\n### Худшие 3 по MAE",
            #worst3_mae.to_markdown(index=False, floatfmt=".4f")
        ]
        
        return "\n".join(tables) + "\n"
    
    def count_trials(self) -> int:
        return len(self.entries)
    
    def get_best(self):
        if not self.entries:
            return None
        df = pd.DataFrame(self.entries)
        return df.loc[df['mse_test'].idxmin()]
    
    def print_best(self):
        best = self.get_best()
        if best is None:
            return
        
        print(f"\n>>>MSE={best['mse_test']:.4f} MAE={best['mae_test']:.4f} "
              f"seq={int(best['seq_len'])} lbl={int(best['label_len'])} e={int(best['e_layers'])} "
              f"h={int(best['n_heads'])} f={int(best['factor'])}\n")

Overwriting llm_opt_toolkit/optimization_journal.py


In [49]:
%%writefile experiments/ETTh1-grid3_pred_len=168;th_b=8192;hypot.json
[
  {
    "trial": 1,
    "timestamp": "2025-11-05T14:13:33.386135",
    "mae_val": 0.8383050560951233,
    "mse_val": 0.891139805316925,
    "mae_test": 0.49326202273368835,
    "mse_test": 0.3694080710411072,
    "seq_len": 96,
    "label_len": 48,
    "e_layers": 3,
    "n_heads": 8,
    "factor": 5,
    "d_model": 512,
    "d_ff": 2048,
    "d_layers": 1,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация использует среднюю длину входной последовательности (96 часов) и умеренную размерность модели, что подходит для захвата краткосрочных и среднесрочных зависимостей в часовых данных. Неглубокий декодер способствует быстрой генерации прогнозов, используя эффективно извлеченные признаки."
  },
  {
    "trial": 2,
    "timestamp": "2025-11-05T14:20:09.188635",
    "mae_val": 0.48567381501197815,
    "mse_val": 0.3630131185054779,
    "mae_test": 0.3964079022407532,
    "mse_test": 0.23787617683410645,
    "seq_len": 336,
    "label_len": 96,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 3,
    "d_model": 768,
    "d_ff": 3072,
    "d_layers": 2,
    "dropout": 0.2,
    "hypothesis": "Данный крупный сетап нацелен на анализ недельных сезонных паттернов (336 часов ввода) с высокой сложностью модели (d_model=768, 4 слоя энкодера, 2 слоя декодера). Повышенный уровень dropout (0.2) используется для предотвращения переобучения при работе с длинными последовательностями и большим количеством параметров."
  },
  {
    "trial": 3,
    "timestamp": "2025-11-05T14:30:58.209652",
    "mae_val": 0.3431564271450043,
    "mse_val": 0.1862528920173645,
    "mae_test": 0.3331473171710968,
    "mse_test": 0.17399954795837402,
    "seq_len": 720,
    "label_len": 168,
    "e_layers": 6,
    "n_heads": 16,
    "factor": 8,
    "d_model": 512,
    "d_ff": 2048,
    "d_layers": 3,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация предназначена для захвата долгосрочных, месячных зависимостей (720 часов ввода), используя максимальную глубину (6 слоев энкодера) и высокую емкость внимания. Умеренный d_model и dropout используются для балансировки сложности и предотвращения переобучения при работе с очень длинными последовательностями."
  },
  {
    "trial": 4,
    "timestamp": "2025-11-05T14:31:33.908912",
    "mae_val": 0.8040050864219666,
    "mse_val": 0.7671905755996704,
    "mae_test": 0.44883689284324646,
    "mse_test": 0.29831990599632263,
    "seq_len": 48,
    "label_len": 24,
    "e_layers": 2,
    "n_heads": 8,
    "factor": 10,
    "d_model": 256,
    "d_ff": 1024,
    "d_layers": 1,
    "dropout": 0.05,
    "hypothesis": "Данный минималистичный сетап ориентирован на краткосрочное прогнозирование (48 часов ввода) и эффективность, используя наименьшую размерность модели (d_model=256) и минимальное количество слоев. Низкий dropout выбран для максимальной чувствительности к недавним паттернам при минимизации вычислительных затрат."
  },
  {
    "trial": 5,
    "timestamp": "2025-11-05T14:34:56.951254",
    "mae_val": 0.8682805895805359,
    "mse_val": 0.930160403251648,
    "mae_test": 0.46394723653793335,
    "mse_test": 0.3237917721271515,
    "seq_len": 168,
    "label_len": 48,
    "e_layers": 4,
    "n_heads": 8,
    "factor": 5,
    "d_model": 512,
    "d_ff": 3072,
    "d_layers": 2,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация балансирует недельную длину входной последовательности (168 часов) с увеличенной сложностью модели (4 слоя энкодера, большой d_ff=3072) для эффективного захвата недельных паттернов. Умеренный dropout и глубина декодера (2) выбраны для стабильного обучения и хорошей генерализации."
  },
  {
    "trial": 6,
    "timestamp": "2025-11-05T15:33:04.688766",
    "mae_val": 0.367705374956131,
    "mse_val": 0.22592321038246155,
    "mae_test": 0.3315505385398865,
    "mse_test": 0.16558507084846497,
    "seq_len": 720,
    "label_len": 336,
    "e_layers": 3,
    "n_heads": 16,
    "factor": 10,
    "d_model": 768,
    "d_ff": 1024,
    "d_layers": 3,
    "dropout": 0.2,
    "hypothesis": "Конфигурация нацелена на анализ очень долгосрочных зависимостей (720 часов) с максимальной размерностью модели (d_model=768) и высоким уровнем регуляризации (dropout=0.2). Используется максимальная длина стартового токена (336) и максимальный фактор (10) для проверки устойчивости при высокой сложности."
  },
  {
    "trial": 7,
    "timestamp": "2025-11-05T15:38:33.385166",
    "mae_val": 0.6993342638015747,
    "mse_val": 0.6214092969894409,
    "mae_test": 0.40045973658561707,
    "mse_test": 0.25122174620628357,
    "seq_len": 168,
    "label_len": 96,
    "e_layers": 6,
    "n_heads": 16,
    "factor": 5,
    "d_model": 768,
    "d_ff": 2048,
    "d_layers": 2,
    "dropout": 0.05,
    "hypothesis": "Эта конфигурация использует максимальную глубину энкодера (6 слоев) и высокую размерность модели (768), чтобы максимально эффективно извлечь недельные и более краткосрочные паттерны из входной последовательности длиной 168 часов. Низкий dropout (0.05) выбран для минимизации потерь информации при обучении высокоемкой модели."
  },
  {
    "trial": 8,
    "timestamp": "2025-11-05T15:41:43.726551",
    "mae_val": 0.828657865524292,
    "mse_val": 0.8752637505531311,
    "mae_test": 0.48146140575408936,
    "mse_test": 0.3351838290691376,
    "seq_len": 48,
    "label_len": 24,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 3,
    "d_model": 512,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.1,
    "hypothesis": "Конфигурация фокусируется на краткосрочном прогнозировании (48 часов ввода) с использованием повышенной сложности (глубокий энкодер/декодер, 16 голов, большой d_ff), чтобы проверить, способна ли высокая емкость модели улучшить результаты для очень коротких временных рядов. Минимальный фактор (3) и умеренный dropout (0.1) используются для точного захвата немедленных зависимостей."
  },
  {
    "trial": 9,
    "timestamp": "2025-11-05T15:42:34.046940",
    "mae_val": 0.823093593120575,
    "mse_val": 0.8192238807678223,
    "mae_test": 0.3975936472415924,
    "mse_test": 0.2491505891084671,
    "seq_len": 96,
    "label_len": 24,
    "e_layers": 2,
    "n_heads": 8,
    "factor": 8,
    "d_model": 256,
    "d_ff": 2048,
    "d_layers": 1,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация тестирует способность минималистичной модели (d_model=256, 2 слоя энкодера, 1 слой декодера) эффективно извлекать зависимости из входной последовательности средней длины (96 часов). Использование относительно большого d_ff должно компенсировать малый размер d_model."
  },
  {
    "trial": 10,
    "timestamp": "2025-11-05T15:54:57.466995",
    "mae_val": 0.41521313786506653,
    "mse_val": 0.25993233919143677,
    "mae_test": 0.3552573025226593,
    "mse_test": 0.18821638822555542,
    "seq_len": 720,
    "label_len": 48,
    "e_layers": 6,
    "n_heads": 16,
    "factor": 3,
    "d_model": 768,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.2,
    "hypothesis": "Эта максимальная по емкости модель (6 слоев, d_model=768, d_ff=3072) предназначена для агрессивного извлечения долгосрочных зависимостей из 720-часовой последовательности. Использование минимального фактора (3) и высокого dropout (0.2) проверяет эффективность плотного внимания при сильной регуляризации."
  },
  {
    "trial": 11,
    "timestamp": "2025-11-05T15:58:42.406090",
    "mae_val": 0.5020892024040222,
    "mse_val": 0.35555213689804077,
    "mae_test": 0.37847641110420227,
    "mse_test": 0.2187207192182541,
    "seq_len": 336,
    "label_len": 168,
    "e_layers": 4,
    "n_heads": 8,
    "factor": 10,
    "d_model": 512,
    "d_ff": 1024,
    "d_layers": 2,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация предназначена для анализа недельных/двухнедельных паттернов (336 часов) с использованием максимального фактора разреженного внимания (10) и умеренного размера модели, чтобы проверить эффективность экстракции длинных, но разреженных зависимостей."
  },
  {
    "trial": 12,
    "timestamp": "2025-11-05T16:02:30.766910",
    "mae_val": 0.7222349047660828,
    "mse_val": 0.6402153372764587,
    "mae_test": 0.3664668798446655,
    "mse_test": 0.20866729319095612,
    "seq_len": 96,
    "label_len": 48,
    "e_layers": 6,
    "n_heads": 16,
    "factor": 5,
    "d_model": 768,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.05,
    "hypothesis": "Данный ультра-сложный сетап для средней длины ввода (96 часов) тестирует, может ли максимальная емкость модели (глубокий энкодер, большой d_model) компенсировать короткую историю, используя низкий dropout для агрессивного обучения."
  },
  {
    "trial": 13,
    "timestamp": "2025-11-05T16:06:43.014856",
    "mae_val": 0.5343169569969177,
    "mse_val": 0.37626540660858154,
    "mae_test": 0.3367091715335846,
    "mse_test": 0.16934828460216522,
    "seq_len": 336,
    "label_len": 48,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 8,
    "d_model": 768,
    "d_ff": 2048,
    "d_layers": 2,
    "dropout": 0.05,
    "hypothesis": "Эта конфигурация использует 2-недельную историю (336 часов) в сочетании с высокой емкостью модели (d_model=768, 16 голов) и низким уровнем регуляризации для агрессивного обучения и точного извлечения среднесрочных сезонных паттернов."
  },
  {
    "trial": 14,
    "timestamp": "2025-11-05T16:09:02.444269",
    "mae_val": 0.6989176273345947,
    "mse_val": 0.6344751715660095,
    "mae_test": 0.3482370376586914,
    "mse_test": 0.18601401150226593,
    "seq_len": 96,
    "label_len": 24,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 5,
    "d_model": 512,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.2,
    "hypothesis": "Эта конфигурация направлена на улучшение краткосрочного прогнозирования (96 часов ввода) за счет использования глубокой архитектуры с максимальной емкостью FFN (3072) и высоким уровнем dropout, чтобы проверить, может ли избыточная сложность модели эффективно извлекать скрытые зависимости из ограниченной истории."
  },
  {
    "trial": 15,
    "timestamp": "2025-11-05T16:12:32.010874",
    "mae_val": 0.5579916834831238,
    "mse_val": 0.40886515378952026,
    "mae_test": 0.3611898720264435,
    "mse_test": 0.19857865571975708,
    "seq_len": 336,
    "label_len": 24,
    "e_layers": 6,
    "n_heads": 8,
    "factor": 3,
    "d_model": 768,
    "d_ff": 1024,
    "d_layers": 1,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация использует максимальную глубину энкодера (6) и максимальный d_model (768) для извлечения комплексных двухнедельных паттернов (336 часов), но с минимальной инициализацией декодера (24) и неглубоким декодером (1), чтобы проверить, достаточно ли богатого представления энкодера для точного прогнозирования."
  },
  {
    "trial": 16,
    "timestamp": "2025-11-05T16:14:23.934463",
    "mae_val": 0.8296902179718018,
    "mse_val": 0.9026740789413452,
    "mae_test": 0.5492066740989685,
    "mse_test": 0.4354010224342346,
    "seq_len": 48,
    "label_len": 24,
    "e_layers": 3,
    "n_heads": 16,
    "factor": 8,
    "d_model": 512,
    "d_ff": 2048,
    "d_layers": 2,
    "dropout": 0.2,
    "hypothesis": "Этот сетап тестирует способность модели средней емкости (d_model=512, 3 слоя энкодера, 2 слоя декодера) эффективно работать с очень коротким входным окном (48 часов), используя высокий dropout (0.2) и большое количество голов внимания (16) для предотвращения переобучения на малом объеме данных."
  },
  {
    "trial": 17,
    "timestamp": "2025-11-05T16:18:01.122129",
    "mae_val": 0.49428603053092957,
    "mse_val": 0.3527902662754059,
    "mae_test": 0.3594265878200531,
    "mse_test": 0.1934441775083542,
    "seq_len": 336,
    "label_len": 96,
    "e_layers": 6,
    "n_heads": 8,
    "factor": 8,
    "d_model": 256,
    "d_ff": 3072,
    "d_layers": 2,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация тестирует, может ли максимальная глубина энкодера (6 слоев) компенсировать малую размерность модели (d_model=256) при извлечении двухнедельных паттернов (336 часов). Использование большого d_ff (3072) должно увеличить экспрессивность, а фактор 8 позволяет эффективно работать с разреженным вниманием на длинных последовательностях."
  },
  {
    "trial": 18,
    "timestamp": "2025-11-05T16:20:46.902876",
    "mae_val": 0.9135962128639221,
    "mse_val": 1.0036064386367798,
    "mae_test": 0.4969310164451599,
    "mse_test": 0.37123924493789673,
    "seq_len": 96,
    "label_len": 48,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 10,
    "d_model": 768,
    "d_ff": 2048,
    "d_layers": 1,
    "dropout": 0.05,
    "hypothesis": "Конфигурация нацелена на агрессивное краткосрочное прогнозирование (96 часов) с использованием максимальной емкости модели (d_model=768) и минимального dropout. При этом используется максимальный фактор разреженности (10) и неглубокий декодер (1 слой), чтобы проверить, достаточно ли агрессивной настройки внимания для короткой истории."
  },
  {
    "trial": 19,
    "timestamp": "2025-11-05T16:27:02.037471",
    "mae_val": 0.8260221481323242,
    "mse_val": 0.8060671091079712,
    "mae_test": 0.4262866675853729,
    "mse_test": 0.2791563868522644,
    "seq_len": 168,
    "label_len": 96,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 5,
    "d_model": 768,
    "d_ff": 2048,
    "d_layers": 3,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация тестирует максимальную глубину декодера (3 слоя) в сочетании с высокой размерностью модели (d_model=768) для оптимизации недельного прогнозирования (seq_len=168), что может улучшить качество генерации признаков. Используется умеренное количество слоев энкодера для баланса между сложностью и временем обучения."
  },
  {
    "trial": 20,
    "timestamp": "2025-11-05T16:29:28.604257",
    "mae_val": 0.4351305067539215,
    "mse_val": 0.2840036451816559,
    "mae_test": 0.38352170586586,
    "mse_test": 0.22181740403175354,
    "seq_len": 720,
    "label_len": 48,
    "e_layers": 4,
    "n_heads": 8,
    "factor": 8,
    "d_model": 256,
    "d_ff": 1024,
    "d_layers": 1,
    "dropout": 0.05,
    "hypothesis": "Данный минималистичный сетап предназначен для проверки, может ли Informer эффективно обрабатывать очень длинные последовательности (720 часов), используя минимальную размерность модели (d_model=256) и неглубокий декодер, полагаясь на высокую разреженность внимания (factor=8) и низкий dropout."
  },
  {
    "trial": 21,
    "timestamp": "2025-11-05T16:35:19.537015",
    "mae_val": 0.9039735198020935,
    "mse_val": 0.9421115517616272,
    "mae_test": 0.509093165397644,
    "mse_test": 0.3956725299358368,
    "seq_len": 168,
    "label_len": 96,
    "e_layers": 6,
    "n_heads": 8,
    "factor": 10,
    "d_model": 768,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.2,
    "hypothesis": "Эта конфигурация максимизирует емкость модели (глубина 6/3, d_model=768, d_ff=3072) для агрессивного извлечения сложных недельных паттернов (168 часов). Высокий dropout (0.2) и максимальный фактор разреженности (10) используются для обеспечения устойчивости и эффективной обработки длинных зависимостей."
  },
  {
    "trial": 22,
    "timestamp": "2025-11-05T16:38:40.995375",
    "mae_val": 0.4794762134552002,
    "mse_val": 0.3325682580471039,
    "mae_test": 0.40526166558265686,
    "mse_test": 0.2549435794353485,
    "seq_len": 720,
    "label_len": 96,
    "e_layers": 4,
    "n_heads": 8,
    "factor": 5,
    "d_model": 512,
    "d_ff": 1024,
    "d_layers": 2,
    "dropout": 0.05,
    "hypothesis": "Эта конфигурация нацелена на эффективное извлечение долгосрочных зависимостей (720 часов) с помощью умеренно глубокой модели (4/2 слоя) и минимальной размерности FFN (1024). Низкий dropout (0.05) и умеренный фактор (5) используются для максимальной чувствительности к паттернам при относительно низкой вычислительной сложности."
  },
  {
    "trial": 23,
    "timestamp": "2025-11-05T16:42:25.408874",
    "mae_val": 0.4821157455444336,
    "mse_val": 0.33508536219596863,
    "mae_test": 0.41619423031806946,
    "mse_test": 0.2735535204410553,
    "seq_len": 720,
    "label_len": 24,
    "e_layers": 3,
    "n_heads": 8,
    "factor": 5,
    "d_model": 256,
    "d_ff": 3072,
    "d_layers": 2,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация тестирует эффективность минималистичной по размерности модели (d_model=256) при обработке очень длинной истории (720 часов), компенсируя малый размер d_model высокой емкостью FFN и полагаясь на минимальную инициализацию декодера (label_len=24)."
  },
  {
    "trial": 24,
    "timestamp": "2025-11-05T16:44:06.194194",
    "mae_val": 0.7605529427528381,
    "mse_val": 0.7087579369544983,
    "mae_test": 0.40412789583206177,
    "mse_test": 0.26155099272727966,
    "seq_len": 168,
    "label_len": 24,
    "e_layers": 3,
    "n_heads": 8,
    "factor": 3,
    "d_model": 768,
    "d_ff": 2048,
    "d_layers": 1,
    "dropout": 0.2,
    "hypothesis": "Данный сетап использует максимальную размерность модели (768) и минимальный фактор внимания (3) для плотного захвата недельных паттернов (168 часов). Высокий dropout и неглубокий декодер используются для регуляризации сложного энкодера и ускорения прогнозирования."
  },
  {
    "trial": 25,
    "timestamp": "2025-11-05T16:46:52.034838",
    "mae_val": 0.7909469604492188,
    "mse_val": 0.7946986556053162,
    "mae_test": 0.3898372948169708,
    "mse_test": 0.24791356921195984,
    "seq_len": 168,
    "label_len": 24,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 5,
    "d_model": 512,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация исследует эффективность глубокого декодера (3 слоя) и высокой емкости FFN (3072) в сочетании с высокой параллельностью внимания (16 голов) для захвата недельных паттернов (168ч) при минимальном стартовом токене."
  },
  {
    "trial": 26,
    "timestamp": "2025-11-05T16:52:07.044085",
    "mae_val": 0.392183780670166,
    "mse_val": 0.24219843745231628,
    "mae_test": 0.3888546824455261,
    "mse_test": 0.23418128490447998,
    "seq_len": 720,
    "label_len": 96,
    "e_layers": 6,
    "n_heads": 16,
    "factor": 10,
    "d_model": 256,
    "d_ff": 2048,
    "d_layers": 3,
    "dropout": 0.2,
    "hypothesis": "Эта конфигурация проверяет, может ли максимальная глубина (6/3 слоев) и агрессивная регуляризация (0.2) компенсировать минимальный размер модели (d_model=256) при моделировании очень длинных последовательностей (720ч) с максимальной разреженностью внимания."
  },
  {
    "trial": 27,
    "timestamp": "2025-11-05T17:02:43.343499",
    "mae_val": 0.4326220750808716,
    "mse_val": 0.2675260603427887,
    "mae_test": 0.3522462248802185,
    "mse_test": 0.19106750190258026,
    "seq_len": 720,
    "label_len": 96,
    "e_layers": 4,
    "n_heads": 16,
    "factor": 3,
    "d_model": 768,
    "d_ff": 2048,
    "d_layers": 3,
    "dropout": 0.05,
    "hypothesis": "Данная конфигурация использует максимальную размерность модели и глубокий декодер, обученные на максимальной истории (720ч). Плотное внимание (factor=3) и минимальный dropout (0.05) нацелены на агрессивный захват комплексных долгосрочных зависимостей."
  },
  {
    "trial": 28,
    "timestamp": "2025-11-05T17:04:33.482089",
    "mae_val": 0.5494969487190247,
    "mse_val": 0.40723511576652527,
    "mae_test": 0.35991916060447693,
    "mse_test": 0.1993093192577362,
    "seq_len": 336,
    "label_len": 168,
    "e_layers": 6,
    "n_heads": 8,
    "factor": 5,
    "d_model": 256,
    "d_ff": 3072,
    "d_layers": 1,
    "dropout": 0.2,
    "hypothesis": "Эта конфигурация исследует, может ли максимальная глубина энкодера (6 слоев) компенсировать минимальный d_model (256) для извлечения двухнедельных паттернов, используя широкий FFN и высокий dropout для регуляризации."
  },
  {
    "trial": 29,
    "timestamp": "2025-11-05T17:12:36.090913",
    "mae_val": 0.36136186122894287,
    "mse_val": 0.20471155643463135,
    "mae_test": 0.32098788022994995,
    "mse_test": 0.15742142498493195,
    "seq_len": 336,
    "label_len": 168,
    "e_layers": 6,
    "n_heads": 16,
    "factor": 3,
    "d_model": 768,
    "d_ff": 3072,
    "d_layers": 2,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация максимизирует модельную емкость (d_model=768, e_layers=6) и использует плотное внимание (factor=3), чтобы агрессивно захватить комплексные, высокоточные зависимости на двухнедельном временном горизонте (336 часов)."
  },
  {
    "trial": 30,
    "timestamp": "2025-11-05T17:19:20.469157",
    "mae_val": 0.3954492211341858,
    "mse_val": 0.24077993631362915,
    "mae_test": 0.4042089283466339,
    "mse_test": 0.23727548122406006,
    "seq_len": 720,
    "label_len": 336,
    "e_layers": 2,
    "n_heads": 8,
    "factor": 8,
    "d_model": 256,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.05,
    "hypothesis": "Данный сетап тестирует способность минималистичного энкодера (d_model=256, e_layers=2) обрабатывать очень длинные последовательности (720 часов), компенсируя малый размер широким FFN (3072), глубоким декодером и минимальным уровнем регуляризации."
  }
]

Writing experiments/ETTh1-grid3_pred_len=168;th_b=8192;hypot.json


In [51]:
from kaggle_secrets import UserSecretsClient
import os
import subprocess

user_secrets = UserSecretsClient()
GEMINI_KEY = user_secrets.get_secret("GEMINI_KEY")
os.environ['GEMINI_API_KEY'] = GEMINI_KEY

pred_len = 168
grid_name = "grid3"
dataset = 'ETTh1'

result = subprocess.run(
    ['tail', '-n', '25', f"experiments/{dataset}-{grid_name}_pred_len={pred_len};th_b=8192;hypot.json"],
    capture_output=True, text=True
)
print(result.stdout)

    "d_model": 768,
    "d_ff": 3072,
    "d_layers": 2,
    "dropout": 0.1,
    "hypothesis": "Эта конфигурация максимизирует модельную емкость (d_model=768, e_layers=6) и использует плотное внимание (factor=3), чтобы агрессивно захватить комплексные, высокоточные зависимости на двухнедельном временном горизонте (336 часов)."
  },
  {
    "trial": 30,
    "timestamp": "2025-11-05T17:19:20.469157",
    "mae_val": 0.3954492211341858,
    "mse_val": 0.24077993631362915,
    "mae_test": 0.4042089283466339,
    "mse_test": 0.23727548122406006,
    "seq_len": 720,
    "label_len": 336,
    "e_layers": 2,
    "n_heads": 8,
    "factor": 8,
    "d_model": 256,
    "d_ff": 3072,
    "d_layers": 3,
    "dropout": 0.05,
    "hypothesis": "Данный сетап тестирует способность минималистичного энкодера (d_model=256, e_layers=2) обрабатывать очень длинные последовательности (720 часов), компенсируя малый размер широким FFN (3072), глубоким декодером и минимальным уровнем регуляризации."
  }
]



In [52]:
subprocess.run([
    "python", "optimize.py",
    "--dataset", f"data/{dataset}_prepared.csv",
    "--pred_len", str(pred_len),
    "--journal", f"{dataset}-{grid_name}_pred_len={pred_len};th_b=8192;hypot.json",
    "--n_batches", "3",
    "--batch_size", "2",
    "--temperature", "1",
    "--thinking_budget", "8192",
    "--grid_name", grid_name
])


PRED_LEN=168ч | 2025-11-17 21:19:45.466066

Attempting to call LLM, try 1
[31/36] seq_len=168 label_len=96 e_layers=6 n_heads=16 factor=3 d_model=512 d_ff=3072 d_layers=3 dropout=0.05 Use GPU: cuda:0
train 11859
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.1857644
	speed: 0.2949s/iter; left time: 625.5272s
	iters: 200, epoch: 1 | loss: 0.2285168
	speed: 0.2797s/iter; left time: 565.3697s
	iters: 300, epoch: 1 | loss: 0.1545940
	speed: 0.2892s/iter; left time: 555.4618s
Epoch: 1 cost time: 106.00029802322388
Epoch: 1, Steps: 370 | Train Loss: 0.2238136 Vali Loss: 0.8656921 Test Loss: 0.3196433
Validation loss decreased (inf --> 0.865692).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1142543
	speed: 0.6839s/iter; left time: 1197.4987s
	iters: 200, epoch: 2 | loss: 0.1047616
	speed: 0.3039s/iter; left time: 501.6721s
	iters: 300, epoch: 2 | loss: 0.0841803
	speed: 0.3053s/iter; left time: 473.5923s
Epoch: 2 cost time: 113.00149488449097
Epoch: 

CompletedProcess(args=['python', 'optimize.py', '--dataset', 'data/ETTh1_prepared.csv', '--pred_len', '168', '--journal', 'ETTh1-grid3_pred_len=168;th_b=8192;hypot.json', '--n_batches', '3', '--batch_size', '2', '--temperature', '1', '--thinking_budget', '8192', '--grid_name', 'grid3'], returncode=0)

In [48]:
!ls

 config.py
 data
'ETTh1-grid3_pred_len=24;th_b=8192;hypot.json'
 experiments
 Informer2020
 informer_checkpoints
 llm_opt_toolkit
 nas-informer.ipynb
 optimize.py
'pred_len=24;th_b=8192;hypot.json'
'pred_len=24;th_b=8192;hypot.json.1'
'pred_len=24;th_b=8192;hypot.json.2'
 __pycache__
 pyproject.toml
 README.md
 results
 test_functions.ipynb
 uv.lock
